# SQL Alchemy

## Install

In [1]:
%pip install --upgrade sqlalchemy

     |████████████████████████████████| 5.9MB 871kB/s eta 0:00:01
  Stored in directory: /home/hsm/.cache/pip/wheels/f4/74/97/23789e1e541fb3d64dc9a921a676539695b3d184cf77f5b009
Successfully built sqlalchemy
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sqlalchemy

In [3]:
sqlalchemy.__version__

'1.3.5'

In [9]:
engine = sqlalchemy.create_engine('sqlite:///create.db', echo=True)

In [10]:
engine

Engine(sqlite:///create.db)

In [11]:
type(engine)

sqlalchemy.engine.base.Engine

In [12]:
metadata = sqlalchemy.MetaData()

In [14]:
users = sqlalchemy.Table('users', metadata,
                         sqlalchemy.Column('id', sqlalchemy.Integer, primary_key=True),
                         sqlalchemy.Column('name', sqlalchemy.String),
                         sqlalchemy.Column('fullname', sqlalchemy.String)
)

In [15]:
addresses = sqlalchemy.Table('addresses', metadata,
                             sqlalchemy.Column('id', sqlalchemy.Integer, primary_key=True),
                             sqlalchemy.Column('user_id', None, sqlalchemy.ForeignKey('users.id')),
                             sqlalchemy.Column('email_address', sqlalchemy.String, nullable=False)
)

In [16]:
metadata.create_all(engine)

2019-07-09 02:14:24,834 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-09 02:14:24,843 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 02:14:24,845 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-09 02:14:24,851 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 02:14:24,854 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-07-09 02:14:24,856 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 02:14:24,859 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2019-07-09 02:14:24,863 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 02:14:24,865 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2019-07-09 02:14:24,871 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 02:14:24,874 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 02:14:24,877 INFO sqlalchemy.engine.b

In [17]:
metadata.tables

immutabledict({'users': Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), schema=None), 'addresses': Table('addresses', MetaData(bind=None), Column('id', Integer(), table=<addresses>, primary_key=True, nullable=False), Column('user_id', Integer(), ForeignKey('users.id'), table=<addresses>), Column('email_address', String(), table=<addresses>, nullable=False), schema=None)})

In [18]:
metadata.bind

In [20]:
print(users.insert())

INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)


In [21]:
insert = users.insert()

In [25]:
insert = insert.values(name='Test', fullname='Test User')

In [26]:
print(insert)

INSERT INTO users (name, fullname) VALUES (:name, :fullname)


In [27]:
insert.compile().params

{'name': 'Test', 'fullname': 'Test User'}

In [40]:
print(users.select())

SELECT users.id, users.name, users.fullname 
FROM users


In [41]:
select = users.select()

In [42]:
select.compile().params

{}

In [33]:
# conn.close()

In [34]:
conn = engine.connect()

In [35]:
insert.bind = engine

In [37]:
print(insert)

INSERT INTO users (name, fullname) VALUES (?, ?)


In [38]:
result = conn.execute(insert)

2019-07-09 02:32:43,892 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 02:32:43,900 INFO sqlalchemy.engine.base.Engine ('Test', 'Test User')
2019-07-09 02:32:43,903 INFO sqlalchemy.engine.base.Engine COMMIT


In [39]:
result.inserted_primary_key

[1]

In [43]:
select = users.select()

In [44]:
select.bind = engine

In [46]:
print(select)

SELECT users.id, users.name, users.fullname 
FROM users


In [47]:
result = conn.execute(select)

2019-07-09 02:34:25,375 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-07-09 02:34:25,378 INFO sqlalchemy.engine.base.Engine ()


In [53]:
result.fetchall()

[(1, 'Test', 'Test User')]

In [55]:
insert = users.insert()
result = conn.execute(insert,
            name='NAME',
            fullname='FULL NAME')
result.inserted_primary_key

2019-07-09 02:38:03,788 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 02:38:03,794 INFO sqlalchemy.engine.base.Engine ('NAME', 'FULL NAME')
2019-07-09 02:38:03,797 INFO sqlalchemy.engine.base.Engine COMMIT


[2]

In [56]:
conn.execute(addresses.insert(), [
    {
        'user_id': 1,
        'email_address': 'test@test.com'
    },
    {
        'user_id': 2,
        'email_address': 'test2@test2.com'
    }
])

2019-07-09 02:39:38,471 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)
2019-07-09 02:39:38,476 INFO sqlalchemy.engine.base.Engine ((1, 'test@test.com'), (2, 'test2@test2.com'))
2019-07-09 02:39:38,484 INFO sqlalchemy.engine.base.Engine COMMIT


In [57]:
sql = sqlalchemy.sql.select([users,])

In [59]:
print(sql)

SELECT users.id, users.name, users.fullname 
FROM users


In [64]:
result = conn.execute(sql)

2019-07-09 02:44:06,677 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-07-09 02:44:06,683 INFO sqlalchemy.engine.base.Engine ()


In [62]:
for row in result:
    print(row)

(1, 'Test', 'Test User')
(2, 'NAME', 'FULL NAME')


In [66]:
result = conn.execute(sql)

2019-07-09 02:44:26,354 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-07-09 02:44:26,357 INFO sqlalchemy.engine.base.Engine ()


In [67]:
result.fetchall()

[(1, 'Test', 'Test User'), (2, 'NAME', 'FULL NAME')]

---

In [71]:
print(users.c.id == addresses.c.user_id)

users.id = addresses.user_id


In [74]:
print(
    sqlalchemy.or_(
        users.c.id == addresses.c.user_id,
        users.c.id == 1
    )
)

users.id = addresses.user_id OR users.id = :id_1


In [76]:
print(sqlalchemy.select([users]).where(users.c.id == 1))

SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.id = :id_1


---

In [77]:
metadata = sqlalchemy.MetaData()

In [78]:
metadata.tables

immutabledict({})

In [85]:
engine = sqlalchemy.create_engine('sqlite:///create6.db', echo=True)

In [86]:
engine

Engine(sqlite:///create6.db)

In [87]:
conn = engine.connect()

2019-07-09 04:24:01,221 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-09 04:24:01,225 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 04:24:01,226 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-09 04:24:01,229 INFO sqlalchemy.engine.base.Engine ()


In [88]:
metadata = sqlalchemy.MetaData()

In [89]:
mytable = sqlalchemy.Table('mytable', metadata,
    sqlalchemy.Column('id', sqlalchemy.Integer, primary_key=True),
    sqlalchemy.Column('value', sqlalchemy.String)
)

In [92]:
print(sqlalchemy.select([mytable]))

SELECT mytable.id, mytable.value 
FROM mytable


In [96]:
metadata.create_all(engine)

2019-07-09 04:26:43,984 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("mytable")
2019-07-09 04:26:43,984 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 04:26:43,990 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE mytable (
	id INTEGER NOT NULL, 
	value VARCHAR, 
	PRIMARY KEY (id)
)


2019-07-09 04:26:43,992 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 04:26:43,996 INFO sqlalchemy.engine.base.Engine COMMIT


In [107]:
insert = sqlalchemy.insert(mytable)

In [110]:
conn.execute(insert.values(value='Hello'))

2019-07-09 04:28:55,513 INFO sqlalchemy.engine.base.Engine INSERT INTO mytable (value) VALUES (?)
2019-07-09 04:28:55,519 INFO sqlalchemy.engine.base.Engine ('Hello',)
2019-07-09 04:28:55,524 INFO sqlalchemy.engine.base.Engine COMMIT


In [111]:
resultset = conn.execute(sqlalchemy.select([mytable]))

2019-07-09 04:29:08,056 INFO sqlalchemy.engine.base.Engine SELECT mytable.id, mytable.value 
FROM mytable
2019-07-09 04:29:08,061 INFO sqlalchemy.engine.base.Engine ()


In [112]:
resultset.fetchall()

[(1, 'Hello')]

In [114]:
sql = sqlalchemy.select([users, addresses]).where(users.c.id == addresses.c.user_id)

In [116]:
print(sql)

SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id


In [118]:
print(users.join(addresses))

users JOIN addresses ON users.id = addresses.user_id
